In [14]:
import cv2 as cv
import dlib
import numpy as np
from ultralytics import YOLO
import supervision as sv
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dist
from pathlib import Path

MODEL_PATH = r"C:\Users\user\Downloads\iran\FineTune.pt"
PERSON_PICTURE_DIR = r"C:\Users\user\Downloads\iran\Hamid\Hamid"

FRAME_WIDTH = 640
FRAME_HEIGHT = 480

RECOGNITION_THRESHOLD = 0.6
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 3


PREDICTOR_PATH = r"C:\Users\user\Downloads\iran\Face_Landmarks"
FACE_REC_MODEL_PATH = r"C:\Users\user\Downloads\iran\dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(PREDICTOR_PATH) #68
facerec = dlib.face_recognition_model_v1(FACE_REC_MODEL_PATH)#128

(L_START, L_END) = (42, 48)
(R_START, R_END) = (36, 42)


def calculate_ear(eye):
    A = dist.euclidean(eye[1], eye[5])# measuring eye points 
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def load_reference_embeddings(person_dir): #create a uniqe ebmedding 
    reference_embeddings = {}
    if detector is None or sp is None or facerec is None:
        return reference_embeddings

    person_path = Path(person_dir)
    if not person_path.exists():
        return reference_embeddings

    person_name = person_path.name 
    person_embs = []
    
    image_files = list(person_path.glob("*.[jp][pn]g")) + list(person_path.glob("*.jpeg"))
    
    for img_path in image_files:
        img = cv.imread(str(img_path))
        if img is None: continue

        rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        faces = detector(rgb_img, 1)
        
        if len(faces) > 0:
            landmarks = sp(rgb_img, faces[0])
            embedding = np.array(facerec.compute_face_descriptor(rgb_img, landmarks))
            person_embs.append(embedding)
            
    if person_embs:
        reference_embeddings[person_name] = person_embs
    
    return reference_embeddings






def main():
    
    cap = cv.VideoCapture(0)
     
    cap.set(cv.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    model = YOLO(MODEL_PATH)
    reference_embeddings = load_reference_embeddings(PERSON_PICTURE_DIR)

    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_scale=0.7, text_thickness=2, text_padding=4)
    tracker = sv.ByteTrack()

    liveness_state = {}

    while True:
        ret, frame = cap.read()
        if not ret: break

        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        





        # Face Detection
        # ________________________________________________________
        result = model(frame, verbose=False, conf=0.5)[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)

        labels = []

        for i in range(len(detections)):
            xyxy = detections.xyxy[i]
            tracker_id = detections.tracker_id[i]

            if tracker_id not in liveness_state:
                liveness_state[tracker_id] = {
                    "blink_counter": 0, "total_blinks": 0, "name": "Unknown", 
                    "live": False, "is_identified": False
                }
            state = liveness_state[tracker_id]

            padding = 20
            x1, y1, x2, y2 = map(int, xyxy)
            h, w, _ = frame.shape
            x1_pad = max(0, x1 - padding)
            y1_pad = max(0, y1 - padding)
            x2_pad = min(w, x2 + padding)
            y2_pad = min(h, y2 + padding)

            dlib_rect = dlib.rectangle(x1_pad, y1_pad, x2_pad, y2_pad)
            
            try:
                landmarks = sp(rgb_frame, dlib_rect)
                






                # Liveness Detection
                # ---------------------------------------
                left_eye = np.array([[p.x, p.y] for p in landmarks.parts()[L_START:L_END]])
                right_eye = np.array([[p.x, p.y] for p in landmarks.parts()[R_START:R_END]])
                avg_ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0

                if avg_ear < EAR_THRESHOLD:
                    state["blink_counter"] += 1
                else:
                    if state["blink_counter"] >= EAR_CONSEC_FRAMES:
                        state["total_blinks"] += 1
                        state["live"] = True
                    state["blink_counter"] = 0







                # Face Recognition
                # ---------------------------------------
                if not state["is_identified"]:
                    embedding = np.array(facerec.compute_face_descriptor(rgb_frame, landmarks))
                    
                    max_similarity = -1
                    recognized_person = "Unknown"
                    for ref_name, ref_embs_list in reference_embeddings.items():
                        for ref_emb in ref_embs_list:
                            similarity = cosine_similarity(embedding.reshape(1, -1), ref_emb.reshape(1, -1))[0][0]
                            if similarity > max_similarity:
                                max_similarity = similarity
                                recognized_person = ref_name
                    
                    if max_similarity > RECOGNITION_THRESHOLD:
                        state["name"] = recognized_person
                    
                    state["is_identified"] = True
            
            except Exception as e:
                state["name"] = "Face Not Aligned"

            liveness_status = "Live" if state["live"] else "Spoof/Photo"
            label = f"{state['name']} | {liveness_status} (Blinks: {state['total_blinks']})"
            labels.append(label)

        current_tracker_ids = set(detections.tracker_id)
        for tracker_id in list(liveness_state.keys()):
            if tracker_id not in current_tracker_ids:
                del liveness_state[tracker_id]

        frame = box_annotator.annotate(scene=frame, detections=detections)
        frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

        cv.imshow("Face Recognition & Liveness", frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()

In [15]:
import cv2 as cv
import dlib
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import supervision as sv
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dist

MODEL_PATH = r"C:\Users\user\Downloads\iran\FineTune.pt"
PERSON_PICTURE_DIR = r"C:\Users\user\Downloads\iran\Hamid\Hamid"

FRAME_WIDTH = 640
FRAME_HEIGHT = 480

RECOGNITION_THRESHOLD = 0.6
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 3

# Head Pose Estimation parameters (new)
MODEL_POINTS = np.array([
    (0.0, 0.0, 0.0),             # Nose tip
    (0.0, -330.0, -65.0),        # Chin
    (-225.0, 170.0, -135.0),     # Left eye left corner
    (225.0, 170.0, -135.0),      # Right eye right corner
    (-150.0, -150.0, -125.0),    # Left mouth corner
    (150.0, -150.0, -125.0)      # Right mouth corner
], dtype="double")

PREDICTOR_PATH = r"C:\Users\user\Downloads\iran\Face_Landmarks"
FACE_REC_MODEL_PATH = r"C:\Users\user\Downloads\iran\dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(PREDICTOR_PATH)
facerec = dlib.face_recognition_model_v1(FACE_REC_MODEL_PATH)

(L_START, L_END) = (42, 48)
(R_START, R_END) = (36, 42)


def calculate_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def load_reference_embeddings(person_dir):
    reference_embeddings = {}
    
    # Re-adding essential safety checks for model loading and directory existence
    if detector is None or sp is None or facerec is None:
        print("Error: Dlib models not loaded. Cannot load reference embeddings.")
        return reference_embeddings
    
    person_path = Path(person_dir)
    if not person_path.exists():
        print(f"Error: Reference person directory not found: {person_dir}")
        return reference_embeddings
    
    person_name = person_path.name 
    person_embs = []
    
    image_files = list(person_path.glob("*.[jp][pn]g")) + list(person_path.glob("*.jpeg"))
    
    for img_path in image_files:
        img = cv.imread(str(img_path))
        if img is None: # This check is also essential for image loading
            print(f"Warning: Could not load image {img_path}. Skipping.")
            continue

        rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        faces = detector(rgb_img, 1)
        
        # ESSENTIAL FIX: Re-adding the check for detected faces
        if len(faces) > 0:
            landmarks = sp(rgb_img, faces[0]) 
            embedding = np.array(facerec.compute_face_descriptor(rgb_img, landmarks))
            person_embs.append(embedding)
        else:
            print(f"Warning: No face detected in reference image {img_path}. Skipping.")
            
    if person_embs:
        reference_embeddings[person_name] = person_embs
    
    return reference_embeddings

def main():
    # Re-adding essential safety checks for main function setup
    if sp is None or facerec is None:
        print("Error: Dlib shape predictor or face recognition model not available. Exiting.")
        return
        
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam. Exiting.")
        return
        
    cap.set(cv.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    # Initialize camera matrix and distortion coefficients for head pose
    focal_length = FRAME_WIDTH
    center = (FRAME_WIDTH/2, FRAME_HEIGHT/2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype = "double"
    )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    model = YOLO(MODEL_PATH)
    reference_embeddings = load_reference_embeddings(PERSON_PICTURE_DIR)

    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_scale=0.7, text_thickness=2, text_padding=4)
    tracker = sv.ByteTrack()

    liveness_state = {} # Stores blink count, recognition status, and now head pose state

    while True:
        ret, frame = cap.read()
        if not ret: 
            print("Error: Could not read frame from webcam. Exiting loop.")
            break

        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        # Face Detection
        # ________________________________________________________
        result = model(frame, verbose=False, conf=0.5)[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)

        labels = []

        for i in range(len(detections)):
            xyxy = detections.xyxy[i]
            tracker_id = detections.tracker_id[i]

            if tracker_id not in liveness_state:
                liveness_state[tracker_id] = {
                    "blink_counter": 0, 
                    "total_blinks": 0, 
                    "name": "Unknown", 
                    "live": False, 
                    "is_identified": False,
                    "head_pose_moved": False, # New for head pose
                    "head_pose_count": 0      # New for head pose
                }
            state = liveness_state[tracker_id]

            padding = 20
            x1, y1, x2, y2 = map(int, xyxy)
            h, w, _ = frame.shape
            x1_pad = max(0, x1 - padding)
            y1_pad = max(0, y1 - padding)
            x2_pad = min(w, x2 + padding)
            y2_pad = min(h, y2 + padding)

            dlib_rect = dlib.rectangle(x1_pad, y1_pad, x2_pad, y2_pad)
            
            # Re-adding a try-except block here is highly recommended for live processing
            try: 
                landmarks = sp(rgb_frame, dlib_rect)
                
                # Liveness Detection (Blink & Head Pose)
                # ---------------------------------------
                left_eye = np.array([[p.x, p.y] for p in landmarks.parts()[L_START:L_END]])
                right_eye = np.array([[p.x, p.y] for p in landmarks.parts()[R_START:R_END]])
                avg_ear = (calculate_ear(left_eye) + calculate_ear(right_eye)) / 2.0

                # Blink Detection
                if avg_ear < EAR_THRESHOLD:
                    state["blink_counter"] += 1
                else:
                    if state["blink_counter"] >= EAR_CONSEC_FRAMES:
                        state["total_blinks"] += 1
                        # A blink is a strong sign of liveness
                        state["live"] = True 
                    state["blink_counter"] = 0

                # Head Pose Estimation (New Liveness Check)
                image_points = np.array([
                    (landmarks.parts()[30].x, landmarks.parts()[30].y),    # Nose tip
                    (landmarks.parts()[8].x, landmarks.parts()[8].y),      # Chin
                    (landmarks.parts()[36].x, landmarks.parts()[36].y),    # Left eye left corner
                    (landmarks.parts()[45].x, landmarks.parts()[45].y),    # Right eye right corner
                    (landmarks.parts()[48].x, landmarks.parts()[48].y),    # Left mouth corner
                    (landmarks.parts()[54].x, landmarks.parts()[54].y)     # Right mouth corner
                ], dtype="double")

                (success, rotation_vector, translation_vector) = cv.solvePnP(
                    MODEL_POINTS, image_points, camera_matrix, dist_coeffs, flags=cv.SOLVEPNP_ITERATIVE
                )

                rmat, jac = cv.Rodrigues(rotation_vector)
                angles, mtxR, mtxQ, Qx, Qy, Qz = cv.RQDecomp3x3(rmat)
                
                current_yaw = angles[1] 

                if not state["head_pose_moved"]:
                    if abs(current_yaw) > 10: 
                        state["head_pose_moved"] = True
                        state["head_pose_count"] += 1 
                        state["live"] = True 
                
                # Face Recognition
                # ---------------------------------------
                if not state["is_identified"]:
                    embedding = np.array(facerec.compute_face_descriptor(rgb_frame, landmarks))
                    
                    max_similarity = -1
                    recognized_person = "Unknown"
                    for ref_name, ref_embs_list in reference_embeddings.items():
                        for ref_emb in ref_embs_list:
                            similarity = cosine_similarity(embedding.reshape(1, -1), ref_emb.reshape(1, -1))[0][0]
                            if similarity > max_similarity:
                                max_similarity = similarity
                                recognized_person = ref_name
                    
                    if max_similarity > RECOGNITION_THRESHOLD:
                        state["name"] = recognized_person
                    
                    state["is_identified"] = True
            
            except Exception as e:
                # If dlib processing fails (e.g., landmarks not found or solvePnP fails)
                state["name"] = "Face Not Aligned/Processed"
                # print(f"Processing error for tracker_id {tracker_id}: {e}") # Debugging aid

            liveness_status = "Live" if state["live"] else "Spoof/Photo"
            label = f"{state['name']} | {liveness_status} (Blinks: {state['total_blinks']}) (Head: {state['head_pose_count']})" 
            labels.append(label)

        current_tracker_ids = set(detections.tracker_id)
        for tracker_id in list(liveness_state.keys()):
            if tracker_id not in current_tracker_ids:
                del liveness_state[tracker_id]

        frame = box_annotator.annotate(scene=frame, detections=detections)
        frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

        cv.imshow("Face Recognition & Liveness", frame)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()